In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import (
    read_sql_query, 
    DataFrame,
    ExcelWriter,
    options
)
from dataframe_image import export


from enum import Enum
import openpyxl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from IPython.display import display_html


In [2]:
class Period(Enum):
    start = '2022-01-01'
    end='2022-03-31'
    date_du_jour = datetime.today().strftime("%d_%m_%Y")

In [3]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")



query = f'''
SELECT 
    dh.*,
    dhs.data_name,
    MAX(dhs.date) AS entry_date,
    GROUP_CONCAT(DISTINCT dhs.id_group, '') AS id_group,
    COUNT(*) AS quantity
FROM
    caris_db.dream_hivinfos dh
        LEFT JOIN
    dream_hivinfos_session dhs ON dhs.id = dh.id_dream_hivinfos_session
GROUP BY dhs.data_name, dh.id_patient
HAVING (entry_date >= '{Period.start.value}'
    AND entry_date <= '{Period.end.value}')
'''




dhi1 = read_sql_query(query, engine, parse_dates=True)


# close the pool of connection
engine.dispose()

True

In [4]:
dhi1.id_patient.count()
dhi =  dhi1.groupby(['data_name']).count()
dhi.quantity.sum()

2114

2114

In [5]:
dhi_report = dhi[['quantity']]
dhi_report.reset_index(inplace=True)
dhi_report

,data_name,quantity
0,Accy Nadege,18
1,Alexandre Darice,2
2,Alexis Fresgarline,16
3,Alexis wilgens,29
4,Alexis wilguens,21
...,...,...
64,Sandra Morisset,7
65,Yousena Demosthene,31
66,Yvelise Merilan,38
67,alexis wilguens,14


In [6]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")



query = f'''
SELECT 
    dh.*,
    dhs.data_name,
    MAX(dhs.date) AS entry_date,
    GROUP_CONCAT(DISTINCT dhs.id_group, '') AS id_group,
    COUNT(*) AS quantity
FROM
    caris_db.dream_hivinfos dh
        LEFT JOIN
    dream_hivinfos_session dhs ON dhs.id = dh.id_dream_hivinfos_session
GROUP BY dhs.data_name
HAVING (entry_date >= '{Period.start.value}'
    AND entry_date <= '{Period.end.value}')
'''




dhi2 = read_sql_query(query, engine, parse_dates=True)


# close the pool of connection
engine.dispose()

True

In [7]:
Period.end.value

'2022-03-31'

In [8]:
dh_report = dhi2[['data_name','entry_date','quantity']]
dh_report.rename(columns={'quantity':'nombre_infos_perSession'},inplace=True)
dh_report

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_2977/680292962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dh_report.rename(columns={'quantity':'nombre_infos_perSession'},inplace=True)


,data_name,entry_date,nombre_infos_perSession
0,Accy Nadege,2022-03-08,223
1,Alexandre Darice,2022-03-26,4
2,Alexis Fresgarline,2022-03-18,16
3,Alexis wilgens,2022-02-26,29
4,alexis wilguens,2022-03-26,1546
...,...,...,...
59,Saint Fleurant Rosena,2022-02-27,37
60,Saintilias Valery,2022-03-25,59
61,Sandra Morisset,2022-03-26,72
62,Yousena Demosthene,2022-03-27,39


In [9]:
dh_report.data_name = dh_report.data_name.str.lower()
dhi_report.data_name = dhi_report.data_name.str.lower()



df1_styler = dh_report.style.set_table_attributes("style='display:inline'").set_caption('dh report')
df2_styler = dhi_report.style.set_table_attributes("style='display:inline'").set_caption('dhi report')

display_html(f"{df1_styler._repr_html_()}  LA COMPARAISON VISUELLE  {df2_styler._repr_html_()}", raw=True)


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_2977/1114690279.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dh_report.data_name = dh_report.data_name.str.lower()
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_2977/1114690279.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dhi_report.data_name = dhi_report.data_name.str.lower()


,data_name,entry_date,nombre_infos_perSession
0,accy nadege,2022-03-08,223
1,alexandre darice,2022-03-26,4
2,alexis fresgarline,2022-03-18,16
3,alexis wilgens,2022-02-26,29
4,alexis wilguens,2022-03-26,1546
5,benjamin ketlande,2022-03-03,15
6,benjamin kettelande,2022-03-19,37
7,charlemagne clairvoi,2022-02-12,167
8,charles joseline alexis fresgarline,2022-01-23,5
9,charles joselyne,2022-03-18,105


In [10]:
report_dhi = dhi_report.merge(dh_report, on='data_name',how='left')
report_dhi = report_dhi[['data_name', 'quantity','nombre_infos_perSession','entry_date']]
report_dhi.rename(columns={
        'quantity':'nombre_agyw_perSession' 
    },
        inplace = True
)
report_dhi.fillna('---',inplace=True)

In [11]:
from itertools import chain

In [12]:

hide_rows = list(
    chain(
        report_dhi[0:17].index,
        report_dhi[18:24].index,
        [25],
        report_dhi[27:61].index,
        [62]
    )
)

In [13]:
caris_clerc = report_dhi.style.hide(hide_rows,axis="index")
caris_clerc
export(caris_clerc,'temp_dhi.png')

,data_name,nombre_agyw_perSession,nombre_infos_perSession,entry_date
17,delva claude,132,440.000000,2022-03-27
24,duthlie pierre,71,4345.000000,2022-03-26
26,ernst belabe,26,717.000000,2022-02-11
61,rene modeline,8,8.000000,2022-02-12
63,saintilias valery,43,59.000000,2022-03-25
64,sandra morisset,7,72.000000,2022-03-26
65,yousena demosthene,31,39.000000,2022-03-27
66,yvelise merilan,38,38.000000,2022-03-28
67,alexis wilguens,14,1546.000000,2022-03-26
68,claude delva,34,172.000000,2022-03-27


/usr/local/lib/python3.10/site-packages/dataframe_image/_pandas_accessor.py:69: FutureWarning: this method is deprecated in favour of `Styler.to_html()`
  html = '<div>' + obj.render() + '</div>'
objc[3193]: Class WebSwapCGLLayer is implemented in both /System/Library/Frameworks/WebKit.framework/Versions/A/Frameworks/WebCore.framework/Versions/A/Frameworks/libANGLE-shared.dylib (0x7ffa49f5c318) and /Applications/Google Chrome.app/Contents/Frameworks/Google Chrome Framework.framework/Versions/100.0.4896.75/Libraries/libGLESv2.dylib (0x10b7fdf48). One of the two will be used. Which one is undefined.
[0410/093449.828239:INFO:headless_shell.cc(659)] Written to file /var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/tmp04unolsx/temp.png.


In [14]:
report_dhi.to_excel('rapport_dhi.xlsx',index=False,na_rep="")
caris_clerc.to_excel('rapport_temp.xlsx',index=False, na_rep='---')